### 2. Sometimes, the features (variables, fields) are not given in a dataset but can be created from it; this is known as feature engineering. For example, the original dataset has several clicks done by the same user, so grouping data by user_id will allow you to create new features for each user:

In [1]:
#Import libraries
import pandas as pd

In [2]:
#Import csv dataset
movies = pd.read_csv('vodclickstream_uk_movies_03.csv', sep=',')
#See the dataset
movies.head()

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
0,58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,58777,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287


In [3]:
#Looking at only the entries with Duration not equal to zero
movies2 = movies[movies['duration']>0]
#See the dataset
movies2.head()

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
2,58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
6,58779,2017-01-01 19:43:06,4903.0,The Water Diviner,"Drama, History, War",2014-12-26,7165c2fc94,8e1be40e32
8,58781,2017-01-01 19:46:24,3845.0,Ratter,"Drama, Horror, Thriller",2016-02-12,c39aae36c3,cff8ea652a
11,58784,2017-01-01 20:55:46,6175.0,28 Days,"Comedy, Drama",2000-04-14,584bffaf5f,759ae2eac9


In [4]:
movies2['title'].value_counts().nlargest(5)

title
Black Mirror: Bandersnatch    2638
Bright                        1763
Avengers: Age of Ultron       1442
Annihilation                  1396
Bird Box                      1344
Name: count, dtype: int64

### *a) Favorite genre (i.e., the genre on which the user spent the most time)*

In [5]:
#Create a subset of original dataframe for create column 'Genres Favorite' for each 'user_id'
selected_movies2 = movies2[['user_id', 'genres']]
#This line fix the warning of pandas package
selected_movies2 = selected_movies2.copy()
selected_movies2['genres'] = selected_movies2['genres'].str.split(',')
df3 = selected_movies2.explode('genres')
df3['genres'] = df3['genres'].str.strip()
genre_counts = df3.groupby(by='user_id').value_counts().to_frame().reset_index()
genre_counts = genre_counts[genre_counts['genres']!='NOT AVAILABLE']


fav_genres = pd.DataFrame()
users = list(set(genre_counts['user_id']))

for i in users:
    # Find the maximum value in the 'count' column
    mini = genre_counts[genre_counts['user_id']==i]

    # Select rows with the maximum value in the 'count' column
    max_valuecount = mini['count'].max()

    if max_valuecount > 1:
        result_df = mini[mini['count'] == max_valuecount].nlargest(10, 'count')
        result = result_df.groupby('user_id').agg({'genres': ','.join}).reset_index()
        fav_genres = pd.concat([fav_genres, result])
    else:
        pass

# Rename the 'date' column to 'date_string' in-place
fav_genres.rename(columns={'genres': 'fav_genres'}, inplace=True)
fav_genres.head()

,user_id,fav_genres
0,80859114bd,"Comedy,Drama"
0,7c311b6f91,Comedy
0,1d655c42c9,Comedy
0,852d269211,"Action,Thriller"
0,c5a5a5811a,"Crime,Biography,History,Documentary,Sport"


### *b) Average click duration*

In [6]:
#Create a subset of original dataframe for create column 'Duration Average' for each 'user_id'
selected_movies = movies2[['user_id', 'duration']]
duration_df = selected_movies.groupby(['user_id']).mean()
duration_df.rename(columns={"duration": "duration_avg"}, inplace=True)
duration_df = duration_df.reset_index()
duration_df.head()

,user_id,duration_avg
0,000052a0a0,3470.0
1,000296842d,19327.0
2,0002aab109,83625.0
3,0005c8fbac,473682.5
4,0005d9a8f4,14394.0


### *c) Time of the day (Morning/Afternoon/Night) when the user spends the most time on the platform (the time spent is tracked through the duration of the clicks)*

In [7]:
#Function to create the 'time_of_day' decision
def time_of_day(hour):
    if 4 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 20:
        return 'Afternoon'
    else:
        return 'Night'
    

#Datetime format
datetime_format = '%Y-%m-%d %H:%M:%S'

#Create a subset of original dataframe for create column 'Time of the Day' (Morning/Afternoon/Night) when the user spends the most time on the platform for each 'user_id'
selected_movies3 = movies2[['user_id', 'datetime']]
#This line fix the warning of pandas package
selected_movies3 = selected_movies3.copy()
selected_movies3['datetime'] = pd.to_datetime(selected_movies3['datetime'], format=datetime_format)
#Create 'TimeOfDay' column and insert the right value for each row
selected_movies3['fav_timeofday'] = selected_movies3['datetime'].apply(lambda x: time_of_day(x.hour))
selected_movies3['Frequency'] = selected_movies3.groupby(['user_id', 'fav_timeofday'])['datetime'].transform('count')

fav_timeofday = pd.DataFrame()
users = list(set(selected_movies3['user_id']))

for i in users:
    # Find the maximum value in the 'count' column
    mini = selected_movies3[selected_movies3['user_id']==i]

    # Select rows with the maximum value in the 'count' column
    max_valuecount = mini['Frequency'].max()

    if max_valuecount > 1:
        result_df = mini[mini['Frequency'] == max_valuecount].nlargest(10, 'Frequency')
        result_df = result_df.drop_duplicates(subset=['fav_timeofday'])
        result = result_df.groupby('user_id').agg({'fav_timeofday': ','.join}).reset_index()
        fav_timeofday = pd.concat([fav_timeofday, result])
    else:
        pass

fav_timeofday.head()

,user_id,fav_timeofday
0,7c311b6f91,Afternoon
0,1d655c42c9,Night
0,852d269211,"Afternoon,Night"
0,03b161ae49,Night
0,b31f8f2932,Afternoon


### *d) Is the user an old movie lover, or is he into more recent stuff (content released after 2010)?*

In [8]:
#Function to create the 'time_of_day' decision
def era_film(year):
    if year < 2010:
        return 'Old'
    else:
        return 'New'

#Create a subset of original dataframe for create column 'User Movie Lover' (old or new film) for each 'user_id'
selected_movies4 = movies2[['user_id', 'release_date']]

#Date format
date_format = '%Y-%m-%d'

#This line fix the warning of pandas package
selected_movies4 = selected_movies4.copy()
selected_movies4 = selected_movies4[selected_movies4['release_date']!='NOT AVAILABLE']
selected_movies4['release_date'] = pd.to_datetime(selected_movies4['release_date'], format=date_format)
#Create 'TimeOfDay' column and insert the right value for each row
selected_movies4['fav_filmera'] = selected_movies4['release_date'].apply(lambda x: era_film(x.year))
selected_movies4['Frequency'] = selected_movies4.groupby(['user_id', 'fav_filmera'])['release_date'].transform('count')


fav_filmera = pd.DataFrame()
users = list(set(selected_movies4['user_id']))

for i in users:
    # Find the maximum value in the 'count' column
    mini = selected_movies4[selected_movies4['user_id']==i]

    # Select rows with the maximum value in the 'count' column
    max_valuecount = mini['Frequency'].max()

    if max_valuecount > 1:
        result_df = mini[mini['Frequency'] == max_valuecount].nlargest(2, 'Frequency')
        result_df = result_df.drop_duplicates(subset=['fav_filmera'])
        result = result_df.groupby('user_id').agg({'fav_filmera': ','.join}).reset_index()
        fav_filmera = pd.concat([fav_filmera, result])
    else:
        pass

fav_filmera.head()


,user_id,fav_filmera
0,1d655c42c9,New
0,852d269211,Old
0,c5a5a5811a,New
0,03b161ae49,New
0,b31f8f2932,New


### *e) Average time spent a day by the user (considering only the days he logs in)*

In [9]:
#Create a subset of original dataframe for create column 'Duration Average per day' for each 'user_id'
selected_movies5 = movies2[['user_id', 'duration', 'datetime']]

#Date format
date_format = '%Y-%m-%d'

#This line fix the warning of pandas package
selected_movies5 = selected_movies5.copy()
selected_movies5 = selected_movies5[selected_movies5['datetime']!='NOT AVAILABLE']
selected_movies5['datetime'] = pd.to_datetime(selected_movies5['datetime'], format=datetime_format)
selected_movies5['datetime'] = selected_movies5['datetime'].apply(lambda x: x.date())
#Create 'TimeOfDay' column and insert the right value for each row
duration_perday_df = selected_movies5.groupby(['user_id', 'datetime'])['duration'].mean().reset_index()
duration_perday_df = duration_perday_df.groupby(['user_id'])['duration'].mean().reset_index()
duration_perday_df.rename(columns={"duration": "duration_perday_avg"}, inplace=True)
duration_perday_df.head()

,user_id,duration_perday_avg
0,000052a0a0,3989.395833
1,000296842d,17198.333333
2,0002aab109,83625.000000
3,0005c8fbac,473682.500000
4,0005d9a8f4,14394.000000


### 3. Consider at least 10 additional features that can be generated for each user_id (you can use chatGPT or other LLM tools for suggesting features to create). Describe each of them and add them to the previous dataset you made (the one with five features). In the end, you should have for each user at least 15 features (5 recommended + 10 suggested by you).

### *1. Sum click duration per user*

In [10]:
#Create a subset of original dataframe for create column 'Duration Sum' for each 'user_id'
selected_movies6 = movies2[['user_id', 'duration']]
duration_sum_df = selected_movies6.groupby(['user_id']).sum()
duration_sum_df.rename(columns={"duration": "duration_sum"}, inplace=True)
duration_sum_df = duration_sum_df.reset_index()
duration_sum_df.head()

,user_id,duration_sum
0,000052a0a0,48580.0
1,000296842d,77308.0
2,0002aab109,83625.0
3,0005c8fbac,947365.0
4,0005d9a8f4,14394.0


### *2. Number of Times Movies Watched*

In [11]:
#Create a subset of original dataframe for create column 'Number of Times Movies Watched' for each 'user_id'
selected_movies7 = movies2[['user_id', 'movie_id']]
ntimes_movies_watched_df = selected_movies7.groupby(['user_id']).count()
ntimes_movies_watched_df.rename(columns={"movie_id": "n_times_movies"}, inplace=True)
ntimes_movies_watched_df = ntimes_movies_watched_df.reset_index()
ntimes_movies_watched_df.head()

,user_id,n_times_movies
0,000052a0a0,14
1,000296842d,4
2,0002aab109,1
3,0005c8fbac,2
4,0005d9a8f4,1


### *3. Number of Movies Watched*

In [12]:
#Create a subset of original dataframe for create column 'Number of Movies Watched' for each 'user_id'
selected_movies8 = movies2[['user_id', 'movie_id']]
selected_movies8 = selected_movies8.drop_duplicates(subset=['user_id', 'movie_id'])
nmovies_watched_df = selected_movies8.groupby(['user_id']).count()
nmovies_watched_df.rename(columns={"movie_id": "n_movies"}, inplace=True)
nmovies_watched_df = nmovies_watched_df.reset_index()
nmovies_watched_df.head()

,user_id,n_movies
0,000052a0a0,9
1,000296842d,1
2,0002aab109,1
3,0005c8fbac,2
4,0005d9a8f4,1


### *4. Number of Movies Re-Watched*

In [13]:
#Create a subset of original dataframe for create column 'Number of Movies Re-Watched' for each 'user_id'
ntimes_rewatched_movies = pd.merge(ntimes_movies_watched_df,nmovies_watched_df)
ntimes_rewatched_movies['n_rewatched'] = ntimes_rewatched_movies['n_times_movies'] - ntimes_rewatched_movies['n_movies']
ntimes_rewatched_movies.head()

,user_id,n_times_movies,n_movies,n_rewatched
0,000052a0a0,14,9,5
1,000296842d,4,1,3
2,0002aab109,1,1,0
3,0005c8fbac,2,2,0
4,0005d9a8f4,1,1,0


### *5. First Watched Movie Date*

In [14]:
#Create a subset of original dataframe for create column 'First Watched Movie Date' for each 'user_id'
selected_movies9 = movies2[['user_id', 'datetime']]
#This line fix the warning of pandas package
selected_movies9 = selected_movies9.copy()

#Datetime format
datetime_format = '%Y-%m-%d %H:%M:%S'
selected_movies9['datetime'] = pd.to_datetime(selected_movies9['datetime'], format=datetime_format)
selected_movies9 = selected_movies9.reset_index()

first_watched_df = selected_movies9.iloc[selected_movies9.groupby(['user_id'])['datetime'].idxmin()].reset_index()
first_watched_df = first_watched_df[['user_id','datetime']]
first_watched_df.rename(columns={"datetime": "datetime_first_movie"}, inplace=True)
first_watched_df.head()

,user_id,datetime_first_movie
0,000052a0a0,2017-05-23 18:41:44
1,000296842d,2018-12-30 22:05:13
2,0002aab109,2017-05-06 19:23:06
3,0005c8fbac,2018-03-23 21:59:46
4,0005d9a8f4,2017-01-23 15:03:37


### *6. Last Watched Movie Date*

In [15]:
#Create a subset of original dataframe for create column 'Last Watched Movie Date' for each 'user_id'
selected_movies10 = movies2[['user_id', 'datetime']]
#This line fix the warning of pandas package
selected_movies10 = selected_movies10.copy()

#Datetime format
datetime_format = '%Y-%m-%d %H:%M:%S'
selected_movies10['datetime'] = pd.to_datetime(selected_movies10['datetime'], format=datetime_format)
selected_movies10 = selected_movies10.reset_index()

last_watched_df = selected_movies10.iloc[selected_movies10.groupby(['user_id'])['datetime'].idxmax()].reset_index()
last_watched_df = last_watched_df[['user_id','datetime']]
last_watched_df.rename(columns={"datetime": "datetime_last_movie"}, inplace=True)
last_watched_df.head()

,user_id,datetime_last_movie
0,000052a0a0,2017-06-26 18:25:42
1,000296842d,2018-12-31 20:06:23
2,0002aab109,2017-05-06 19:23:06
3,0005c8fbac,2018-04-13 19:08:10
4,0005d9a8f4,2017-01-23 15:03:37


### *7. Days Since Last Watched (Inactivity)*

In [16]:
from datetime import datetime

# Calculate the difference between today and the time the user watched his last movie.
today = datetime.now()
inactivity_df = last_watched_df.copy()
inactivity_df['inactivity'] = (today - inactivity_df['datetime_last_movie']).dt.days
inactivity_df.head()

,user_id,datetime_last_movie,inactivity
0,000052a0a0,2017-06-26 18:25:42,2359
1,000296842d,2018-12-31 20:06:23,1805
2,0002aab109,2017-05-06 19:23:06,2409
3,0005c8fbac,2018-04-13 19:08:10,2068
4,0005d9a8f4,2017-01-23 15:03:37,2513


### *8. Favourite Month to Watch Movies*

In [17]:
#Function to create the 'choose_month' decision
def choose_month(month):
    if month == 1:
        return 'January'
    if month == 2:
        return 'February'
    if month == 3:
        return 'March'
    if month == 4:
        return 'April'
    if month == 5:
        return 'May'
    if month == 6:
        return 'June'
    if month == 7:
        return 'July'
    if month == 8:
        return 'August'
    if month == 9:
        return 'September'
    if month == 10:
        return 'October'
    if month == 11:
        return 'November'
    elif month == 12:
        return 'December'
    

#Create a subset of original dataframe for create column 'First Watched Movie Date' for each 'user_id'
selected_movies11 = movies2[['user_id', 'datetime']]
#This line fix the warning of pandas package
selected_movies11 = selected_movies11.copy()

#Datetime format
datetime_format = '%Y-%m-%d %H:%M:%S'
selected_movies11['datetime'] = pd.to_datetime(selected_movies11['datetime'], format=datetime_format)
selected_movies11 = selected_movies11.reset_index()

selected_movies11['fav_month'] = selected_movies11['datetime'].apply(lambda x: choose_month(x.month))
selected_movies11['Frequency'] = selected_movies11.groupby(['user_id','fav_month'])['index'].transform('count')


fav_month_df = pd.DataFrame()
users = list(set(selected_movies11['user_id']))

for i in users:
    # Find the maximum value in the 'count' column
    mini = selected_movies11[selected_movies11['user_id']==i]

    # Select rows with the maximum value in the 'count' column
    max_valuecount = mini['Frequency'].max()

    if max_valuecount > 1:
        result_df = mini[mini['Frequency'] == max_valuecount].nlargest(12, 'Frequency')
        result_df = result_df.drop_duplicates(subset=['fav_month'])
        result = result_df.groupby('user_id').agg({'fav_month': ','.join}).reset_index()
        fav_month_df = pd.concat([fav_month_df, result])
    else:
        pass

fav_month_df.head()

,user_id,fav_month
0,80859114bd,February
0,7c311b6f91,September
0,1d655c42c9,December
0,852d269211,February
0,c5a5a5811a,February


### *9. Favourite Year to Watch Movies*

In [41]:
#Create a subset of original dataframe for create column 'First Watched Movie Date' for each 'user_id'
selected_movies13 = movies2[['user_id', 'datetime']]
#This line fix the warning of pandas package
selected_movies13 = selected_movies13.copy()

#Datetime format
datetime_format = '%Y-%m-%d %H:%M:%S'
selected_movies13['datetime'] = pd.to_datetime(selected_movies13['datetime'], format=datetime_format)
selected_movies13 = selected_movies13.reset_index()

selected_movies13['fav_year'] = selected_movies13['datetime'].apply(lambda x: x.year)
selected_movies13['Frequency'] = selected_movies13.groupby(['user_id','fav_year'])['index'].transform('count')
fav_year_df = selected_movies13.iloc[selected_movies13.groupby(['user_id'])['Frequency'].idxmax()].reset_index()
fav_year_df = fav_year_df[['user_id','fav_year']]
fav_year_df.head()

,user_id,fav_year
0,000052a0a0,2017
1,000296842d,2018
2,0002aab109,2017
3,0005c8fbac,2018
4,0005d9a8f4,2017


### *10. Average time elapsed from film release and user watched (per user_id)*

In [23]:
#Create a subset of original dataframe for create column 'Average time elapsed from film release and user watched' for each 'user_id'
selected_movies12 = movies2[['user_id', 'datetime', 'release_date']]
selected_movies12 = selected_movies12[selected_movies12['release_date']!='NOT AVAILABLE']
#This line fix the warning of pandas package
selected_movies12 = selected_movies12.copy()

#Datetime format
datetime_format = '%Y-%m-%d %H:%M:%S'
#Date format
date_format = '%Y-%m-%d'

selected_movies12['datetime'] = pd.to_datetime(selected_movies12['datetime'], format=datetime_format)
selected_movies12['release_date'] = pd.to_datetime(selected_movies12['release_date'], format=date_format)
selected_movies12['time_elapsed'] = (selected_movies12['datetime'] - selected_movies12['release_date']).dt.days

avg_time_elapsed_df = selected_movies12.groupby(['user_id'])['time_elapsed'].mean()
#avg_time_elapsed_df.rename(columns={"time_elapsed": "avg_time_elapsed"}, inplace=True)
avg_time_elapsed_df = avg_time_elapsed_df.reset_index()
avg_time_elapsed_df.head()

,user_id,time_elapsed
0,000052a0a0,4276.285714
1,000296842d,2.750000
2,0002aab109,1940.000000
3,0005c8fbac,3962.000000
4,0005d9a8f4,427.000000


#### *Merge all feature for one dataset*

In [60]:
df1 = pd.merge(duration_df, fav_genres, on='user_id', how='outer')
df2 = pd.merge(fav_timeofday, fav_filmera, on='user_id', how='outer')
df3 = pd.merge(duration_perday_df, duration_sum_df, on='user_id', how='outer')
df4 = pd.merge(ntimes_rewatched_movies, first_watched_df, on='user_id', how='outer')
df5 = pd.merge(last_watched_df, inactivity_df, on='user_id', how='outer')
df6 = pd.merge(fav_month_df, fav_year_df, on='user_id', how='outer')

df7 = pd.merge(df1, df2, on='user_id', how='outer')
df8 = pd.merge(df3, df4, on='user_id', how='outer')
df9 = pd.merge(df5, df6, on='user_id', how='outer')

df10 = pd.merge(df7, df8, on='user_id', how='outer')
df11 = pd.merge(df9, avg_time_elapsed_df, on='user_id', how='outer')

df = pd.merge(df10, df11, on='user_id', how='outer')

df.head()

,user_id,duration_avg,fav_genres,fav_timeofday,fav_filmera,duration_perday_avg,duration_sum,n_times_movies,n_movies,n_rewatched,datetime_first_movie,datetime_last_movie_x,datetime_last_movie_y,inactivity,fav_month,fav_year,time_elapsed
0,000052a0a0,3470.0,"Action,Comedy,Thriller",Afternoon,Old,3989.395833,48580.0,14,9,5,2017-05-23 18:41:44,2017-06-26 18:25:42,2017-06-26 18:25:42,2359,June,2017,4276.285714
1,000296842d,19327.0,"Thriller,Sci-Fi,Mystery,Drama",Night,New,17198.333333,77308.0,4,1,3,2018-12-30 22:05:13,2018-12-31 20:06:23,2018-12-31 20:06:23,1805,December,2018,2.750000
2,0002aab109,83625.0,NaN,NaN,NaN,83625.000000,83625.0,1,1,0,2017-05-06 19:23:06,2017-05-06 19:23:06,2017-05-06 19:23:06,2409,NaN,2017,1940.000000
3,0005c8fbac,473682.5,NaN,NaN,NaN,473682.500000,947365.0,2,2,0,2018-03-23 21:59:46,2018-04-13 19:08:10,2018-04-13 19:08:10,2068,NaN,2018,3962.000000
4,0005d9a8f4,14394.0,NaN,NaN,NaN,14394.000000,14394.0,1,1,0,2017-01-23 15:03:37,2017-01-23 15:03:37,2017-01-23 15:03:37,2513,NaN,2017,427.000000
